In [ ]:
import numpy as np
import sys
sys.path.append('../..')
import pyemma
from simplex import simplex, product_tica
import matplotlib.pyplot as plt
import matplotlib.colors
%matplotlib inline

In [ ]:
def rp(n=30000, s0=0, m=100):
    T = np.array([[m, 1.],[1, m]])
    T /= T.sum(axis=1)[:, np.newaxis]
    T_cdf = T.cumsum(axis=1)
    s = s0
    ts = []
    for _ in range(n):
        ts.append(s)
        s = np.searchsorted(T_cdf[s, :], np.random.rand())
    return ts

In [ ]:
c = 0.5
R = np.array([[c, -(1.-c**2)**0.5], [(1.-c**2)**0.5,  c]])

In [ ]:
ts_x = rp()
ts_y = rp()
ts_x += np.random.randn(len(ts_x))*.1
ts_y += np.random.randn(len(ts_y))*.1

## no products

In [ ]:
ts_no_prod =  np.vstack((ts_x, ts_y)).T.dot(R)

In [ ]:
plt.hist2d(ts_no_prod[:,0], ts_no_prod[:,1], bins=100, norm=matplotlib.colors.LogNorm());

In [ ]:
tica_no_prod = pyemma.coordinates.tica(data = ts_no_prod, lag=5)
vertices_no_prod = simplex.find_vertices_inner_simplex(tica_no_prod.get_output())

In [ ]:
tica_no_prod.dimension()

In [ ]:
tics_no_prod = np.concatenate(tica_no_prod.get_output())

In [ ]:
plt.hist2d(tics_no_prod[:,0], tics_no_prod[:,1], bins=100, norm=matplotlib.colors.LogNorm());

In [ ]:
P, o = simplex.splash_corner_projection(vertices_no_prod, center=2)
proj = (tics_no_prod-o).dot(P)
proj_vertices = (vertices_no_prod-o).dot(P)
plt.hist2d(proj[:,0], proj[:,1], bins=100, norm=matplotlib.colors.LogNorm());
plt.scatter(proj_vertices[:,0], proj_vertices[:,1], c='r', s=80)

## using products

In [ ]:
tica_1st_level, tica_2nd_level = product_tica.product_tica(ts_no_prod, lag=5, return_stage_1=True, take_max_dim=10)
vertices_2nd_level = simplex.find_vertices_inner_simplex(tica_2nd_level)
tics_2nd_level = np.concatenate(tica_2nd_level.get_output())

In [ ]:
tica_1st_level.dimension()

In [ ]:
tica_2nd_level.dimension()

In [ ]:
tica_2nd_level.eigenvalues

In [ ]:
P, o = simplex.splash_corner_projection(vertices_2nd_level, center=2)
proj = (tics_2nd_level-o).dot(P)
proj_vertices = (vertices_2nd_level-o).dot(P)
plt.hist2d(proj[:,0], proj[:,1], bins=100, norm=matplotlib.colors.LogNorm());
plt.scatter(proj_vertices[:,0], proj_vertices[:,1], c='r', s=80)

In [ ]:
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
s = 10
ax.scatter3D(tics_2nd_level[::s, 0], tics_2nd_level[::s, 1], tics_2nd_level[::s, 2])